In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv('./data/data.csv')

Drop the columns we won't need.

In [7]:
data.drop(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W', 'L', 'SEASON', 'PK', 'PK_2', 'PK_3'], axis=1, inplace=True)

Train-test split.

We will keep data from years 2006, 2015 and 2023 for later testing.

In [10]:
# data.dtypes

In [12]:
data_train = data[(data['YEAR_MVP'] != 2006) & (data['YEAR_MVP'] != 2015) & (data['YEAR_MVP'] != 2023)]

In [39]:
data_test = data[(data['YEAR_MVP'] == 2006) | (data['YEAR_MVP'] == 2015) | (data['YEAR_MVP'] == 2023)]

In [18]:
data_train.drop('YEAR_MVP', axis=1, inplace=True)

C:\Users\dcast\AppData\Local\Temp\ipykernel_9360\2447353752.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.drop('YEAR_MVP', axis=1, inplace=True)
C:\Users\dcast\AppData\Local\Temp\ipykernel_9360\2447353752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop('YEAR_MVP', axis=1, inplace=True)


In [15]:
# data_train

In [16]:
# data_test

X-y split.

In [21]:
X_train = data_train.drop('Share', axis=1)
y_train = data_train['Share']

Scaling.

Train.

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [24]:
X_train_tr = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)

In [26]:
X_train_tr.isna().sum().sum()

0

In [30]:
y_train

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
11422    0.0
11423    0.0
11424    0.0
11425    0.0
11426    0.0
Name: Share, Length: 10477, dtype: float64

Model.

In [32]:
pip install xgboost

                                              0.0/99.7 MB ? eta -:--:--
                                              1.1/99.7 MB 22.4 MB/s eta 0:00:05
                                              2.3/99.7 MB 24.2 MB/s eta 0:00:05
     -                                        4.1/99.7 MB 29.0 MB/s eta 0:00:04
     --                                       5.6/99.7 MB 30.0 MB/s eta 0:00:04
     --                                       7.3/99.7 MB 31.1 MB/s eta 0:00:03
     ---                                      9.1/99.7 MB 32.2 MB/s eta 0:00:03
     ----                                    10.8/99.7 MB 36.4 MB/s eta 0:00:03
     ----                                    12.2/99.7 MB 34.4 MB/s eta 0:00:03
     -----                                   14.0/99.7 MB 36.4 MB/s eta 0:00:03
     -----                                   15.2/99.7 MB 36.4 MB/s eta 0:00:03
     ------                                  15.7/99.7 MB 31.2 MB/s eta 0:00:03
     ------                                  17.

In [36]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

model1 = RandomForestRegressor()
model2 = GradientBoostingRegressor()
model3 = SVR()
model4 = XGBRegressor()

model_pipeline = [model1, model2, model3, model4]
model_names = ['Random Forest Regressor', 'Gradient Boosting Regressor', 'SVR', 'XGBoost Regressor']

scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train_tr, y_train, cv=5))
    scores[model_name] = mean_score
print(scores)

{'Random Forest Regressor': 0.6398181465936121, 'Gradient Boosting Regressor': 0.6284845684784688, 'SVR': 0.12467875127048618, 'XGBoost Regressor': 0.5688937149765636}


In [ ]:
from sklearn.model_selection import GridSearchCV
rf_model = RandomForestRegressor()

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

grid_search.fit(X_train_tr, y_train)

best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

best_rf_model = grid_search.best_estimator_

r2_score_train = best_rf_model.score(X_train_tr, y_train)
print("R^2:", r2_score_train)

Test year 2006.

In [42]:
data_test_2006 = data_test[data_test['YEAR_MVP'] == 2006]
data_test_2006.drop('YEAR_MVP', axis=1, inplace=True)

C:\Users\dcast\AppData\Local\Temp\ipykernel_9360\1784467201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_2006.drop('YEAR_MVP', axis=1, inplace=True)


In [43]:
X_test_2006 = data_test_2006.drop('Share', axis=1)
y_test_2006 = data_test_2006['Share']

In [44]:
X_test_2006_tr = pd.DataFrame(scaler.transform(X_test_2006), index=X_test_2006.index, columns=X_test_2006.columns)

In [49]:
pred_2006 = best_rf_model.predict(X_test_2006_tr)

In [50]:
pred_2006 = pd.Series(pred_2006, index=X_test_2006.index)
pred_2006

3094    0.000013
3095    0.000013
3096    0.000013
3097    0.000013
3098    0.000013
          ...   
3547    0.000014
3548    0.000013
3549    0.000013
3550    0.000013
3551    0.000034
Length: 458, dtype: float64

In [58]:
data_or = pd.read_csv('./data/data.csv')
data_2006 = data_or[data_or['YEAR_MVP'] == 2006]

In [61]:
# data_2006

In [62]:
output_2006 = pd.concat([data_2006, pred_2006], axis=1)

In [65]:
output_2006.sort_values(by=0, ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,0
3514,1495,Tim Duncan,Tim,1610612759,SAS,30.0,80,61,19,0.763,34.8,7.2,14.8,0.484,0.0,0.1,0.400,4.2,6.7,0.629,2.9,8.1,11.0,3.2,2.5,0.9,2.0,0.8,2.7,5.6,18.6,6.5,42.7,52,0,38.6,2005-06,1495_2005-06,105.9,106.6,106.6,96.3,97.1,97.1,9.5,9.5,9.5,0.162,1.28,13.5,0.091,0.239,0.167,10.6,10.6,0.485,0.523,0.274,0.277,89.87,89.21,74.35,89.21,0.172,5188,7.2,14.8,1610612759_2005-06,0.768,1,1,2006,1495_2006,0.026,0.403201
3351,977,Kobe Bryant,Kobe,1610612747,LAL,27.0,80,45,35,0.563,41.0,12.2,27.2,0.450,2.3,6.5,0.347,8.7,10.2,0.850,0.9,4.4,5.3,4.5,3.1,1.8,0.4,1.0,2.9,8.1,35.4,4.0,52.0,4,0,51.9,2005-06,977_2005-06,108.4,109.7,109.7,103.7,105.0,105.0,4.7,4.7,4.7,0.228,1.44,11.5,0.022,0.112,0.067,8.0,8.0,0.491,0.559,0.378,0.384,94.36,93.20,77.67,93.20,0.181,6371,12.2,27.2,1610612747_2005-06,0.549,7,3,2006,977_2006,0.386,0.389372
3361,2544,LeBron James,LeBron,1610612739,CLE,21.0,79,47,32,0.595,42.5,11.1,23.1,0.480,1.6,4.8,0.335,7.6,10.3,0.738,0.9,6.1,7.0,6.6,3.3,1.6,0.8,0.8,2.3,7.5,31.4,3.5,53.6,21,5,51.4,2005-06,2544_2005-06,107.2,108.2,108.2,103.1,104.4,104.4,4.2,3.8,3.8,0.315,2.00,17.7,0.023,0.150,0.086,8.9,8.8,0.515,0.568,0.329,0.336,92.25,91.25,76.05,91.25,0.192,6404,11.1,23.1,1610612739_2005-06,0.610,3,2,2006,2544_2006,0.550,0.372837
3227,1717,Dirk Nowitzki,Dirk,1610612742,DAL,28.0,81,60,21,0.741,38.1,9.3,19.3,0.480,1.4,3.3,0.406,6.7,7.4,0.901,1.4,7.6,9.0,2.8,1.9,0.7,1.0,0.9,2.0,5.9,26.6,6.3,44.8,35,0,43.2,2005-06,1717_2005-06,111.3,111.9,111.9,102.7,103.3,103.3,8.5,8.6,8.6,0.142,1.45,10.3,0.041,0.209,0.127,7.1,7.1,0.515,0.589,0.293,0.301,90.13,89.64,74.70,89.64,0.205,5774,9.3,19.3,1610612742_2005-06,0.732,2,2,2006,1717_2006,0.435,0.340276
3236,2548,Dwyane Wade,Dwyane,1610612748,MIA,24.0,75,48,27,0.640,38.6,9.3,18.8,0.495,0.2,1.0,0.171,8.4,10.7,0.783,1.4,4.3,5.7,6.7,3.6,1.9,0.8,0.9,2.9,7.8,27.2,6.5,48.7,16,2,45.3,2005-06,2548_2005-06,109.9,110.5,110.5,101.0,102.1,102.1,8.8,8.4,8.4,0.312,1.88,19.8,0.040,0.112,0.078,10.5,10.6,0.499,0.577,0.318,0.321,94.36,93.57,77.98,93.57,0.186,5643,9.3,18.8,1610612748_2005-06,0.634,2,1,2006,2548_2006,0.070,0.318586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3273,458,Howard Eisley,Howard,1610612743,DEN,33.0,32,17,15,0.531,12.3,1.1,3.2,0.330,0.4,1.3,0.310,0.6,0.8,0.833,0.1,0.9,1.0,2.2,0.7,0.3,0.1,0.2,1.1,0.8,3.2,-2.1,8.2,0,0,7.6,2005-06,458_2005-06,95.6,93.8,93.8,103.7,101.8,101.8,-8.1,-7.9,-7.9,0.294,3.29,33.8,0.008,0.081,0.044,10.3,10.3,0.393,0.445,0.151,0.158,95.23,97.07,80.89,97.07,0.072,795,1.1,3.2,1610612743_2005-06,0.537,4,1,2006,458_2006,0.000,0.000013
3271,101124,Hakim Warrick,Hakim,1610612763,MEM,23.0,68,42,26,0.618,10.6,1.5,3.4,0.443,0.0,0.0,0.000,1.1,1.7,0.661,0.6,1.5,2.1,0.4,0.8,0.2,0.3,0.5,1.7,1.4,4.1,-1.1,8.0,0,0,7.7,2005-06,101124_2005-06,98.2,96.0,96.0,103.0,101.6,101.6,-4.9,-5.5,-5.5,0.080,0.54,8.2,0.063,0.147,0.105,15.3,15.4,0.443,0.499,0.213,0.221,90.59,92.25,76.88,92.25,0.060,1391,1.5,3.4,1610612763_2005-06,0.598,5,3,2006,101124_2006,0.000,0.000013
3270,2775,Ha Ha,Ha,1610612757,POR,20.0,27,4,23,0.148,7.8,0.7,1.1,0.581,0.0,0.0,0.000,0.3,0.6,0.471,0.5,1.3,1.8,0.0,0.5,0.1,0.3,0.1,1.5,0.6,1.6,-2.1,4.6,0,0,4.3,2005-06,2775_2005-06,99.7,98.8,98.8,112.4,110.2,110.2,-12.

Test year 2015.

In [67]:
data_test_2015 = data_test[data_test['YEAR_MVP'] == 2015]
data_test_2015.drop('YEAR_MVP', axis=1, inplace=True)

C:\Users\dcast\AppData\Local\Temp\ipykernel_9360\1022236191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_2015.drop('YEAR_MVP', axis=1, inplace=True)


In [68]:
X_test_2015 = data_test_2015.drop('Share', axis=1)
y_test_2015 = data_test_2015['Share']

In [69]:
X_test_2015_tr = pd.DataFrame(scaler.transform(X_test_2015), index=X_test_2015.index, columns=X_test_2015.columns)

In [70]:
pred_2015 = best_rf_model.predict(X_test_2015_tr)

In [71]:
pred_2015 = pd.Series(pred_2015, index=X_test_2015.index)
pred_2015

7229    0.000013
7230    0.000013
7231    0.000013
7232    0.000013
7233    0.000060
          ...   
7716    0.000013
7717    0.000013
7718    0.000070
7719    0.000013
7720    0.000013
Length: 492, dtype: float64

In [72]:
data_or = pd.read_csv('./data/data.csv')
data_2015 = data_or[data_or['YEAR_MVP'] == 2015]

In [ ]:
# data_2006

In [73]:
output_2015 = pd.concat([data_2015, pred_2015], axis=1)

In [74]:
output_2015.sort_values(by=0, ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,0
7429,201935,James Harden,James,1610612745,HOU,25.0,81,56,25,0.691,36.8,8.0,18.1,0.440,2.6,6.9,0.375,8.8,10.2,0.868,0.9,4.7,5.7,7.0,4.0,1.9,0.7,1.3,2.6,6.7,27.4,4.1,48.6,21,4,47.9,2014-15,201935_2014-15,107.7,109.2,109.2,101.8,103.5,103.5,5.8,5.7,5.7,0.340,1.76,20.8,0.025,0.126,0.075,11.8,11.8,0.511,0.605,0.308,0.313,99.08,97.61,81.35,97.61,0.184,6056,8.0,18.1,1610612745_2014-15,0.683,2,1,2015,201935_2015,0.720,0.306557
7530,2544,LeBron James,LeBron,1610612739,CLE,30.0,69,50,19,0.725,36.1,9.0,18.5,0.488,1.7,4.9,0.354,5.4,7.7,0.710,0.7,5.3,6.0,7.4,3.9,1.6,0.7,0.7,2.0,6.0,25.3,7.8,46.5,22,2,45.0,2014-15,2544_2014-15,112.2,113.8,113.8,102.4,102.8,102.8,9.8,10.9,10.9,0.366,1.88,22.4,0.022,0.147,0.086,11.9,11.9,0.535,0.577,0.318,0.324,94.92,94.06,78.38,94.06,0.171,4889,9.0,18.5,1610612739_2014-15,0.646,2,1,2015,2544_2015,0.425,0.225316
7664,201939,Stephen Curry,Stephen,1610612744,GSW,27.0,80,67,13,0.838,32.7,8.2,16.8,0.487,3.6,8.1,0.443,3.9,4.2,0.914,0.7,3.6,4.3,7.7,3.1,2.0,0.2,0.5,2.0,3.7,23.8,11.5,44.1,23,0,43.8,2014-15,201939_2014-15,114.2,114.7,114.7,97.2,98.5,98.5,17.0,16.3,16.3,0.358,2.49,26.4,0.022,0.100,0.063,10.6,10.6,0.594,0.638,0.277,0.283,102.39,101.47,84.56,101.47,0.178,5534,8.2,16.8,1610612744_2014-15,0.817,1,1,2015,201939_2015,0.922,0.224121
7258,203076,Anthony Davis,Anthony,1610612740,NOP,22.0,68,39,29,0.574,36.1,9.4,17.6,0.535,0.0,0.2,0.083,5.5,6.8,0.805,2.5,7.7,10.2,2.2,1.4,1.5,2.9,0.8,2.1,5.5,24.4,3.9,51.8,42,0,45.6,2014-15,203076_2014-15,108.4,109.9,109.9,103.3,103.8,103.8,5.1,6.1,6.1,0.109,1.57,9.1,0.072,0.215,0.144,5.8,5.8,0.536,0.591,0.272,0.276,94.28,93.40,77.83,93.40,0.192,4765,9.4,17.6,1610612740_2014-15,0.549,8,5,2015,203076_2015,0.156,0.147336
7640,201566,Russell Westbrook,Russell,1610612760,OKC,26.0,67,40,27,0.597,34.4,9.4,22.0,0.426,1.3,4.3,0.299,8.1,9.8,0.835,1.9,5.4,7.3,8.6,4.4,2.1,0.2,1.3,2.7,6.6,28.1,3.7,52.3,31,11,49.9,2014-15,201566_2014-15,109.1,111.0,111.0,105.0,106.3,106.3,4.1,4.7,4.7,0.430,1.96,21.9,0.051,0.151,0.101,11.2,11.2,0.455,0.536,0.368,0.372,100.68,99.23,82.69,99.23,0.187,4769,9.4,22.0,1610612760_2014-15,0.549,9,2,2015,201566_2015,0.271,0.129648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7418,203960,JaKarr Sampson,JaKarr,1610612755,PHI,22.0,74,17,57,0.230,15.3,2.0,4.7,0.422,0.4,1.7,0.244,0.9,1.3,0.670,0.5,1.7,2.2,1.0,1.0,0.5,0.4,0.4,1.8,1.1,5.2,-4.5,11.0,0,0,10.6,2014-15,203960_2014-15,92.9,91.8,91.8,103.8,104.0,104.0,-11.0,-12.2,-12.2,0.121,1.01,14.2,0.028,0.112,0.068,14.0,14.1,0.467,0.498,0.173,0.178,100.36,100.89,84.07,100.89,0.058,2354,2.0,4.7,1610612755_2014-15,0.220,14,4,2015,203960_2015,0.000,0.000013
7417,2747,JR Smith,JR,1610612739,CLE,29.0,70,38,32,0.543,29.7,4.5,10.8,0.417,2.3,6.1,0.383,0.8,1.0,0.750,0.4,2.7,3.1,2.8,1.4,1.2,0.3,0.2,2.3,1.1,12.1,2.3,23.1,0,0,23.3,2014-15,2747_2014-15,107.9,108.6,108.6,105.1,105.0,105.0,2.8,3.6,3.6,0.145,2.01,18.1,0.015,0.091,0.053,9.0,9.0,0.525,0.538,0.188,0.191,94.91,94.64,78.87,94.64,0.083,4105,4.5,10.8,1610612739_2014-15,0.646,2,1,2015,2747_2015,0.000,0.000013
7415,201581,JJ Hickson,JJ,1610612743,DEN,26.0,73,26,47,0.356,19.3,3.0,6.3,0.475,0.0,0.0,0.000,1.6,2.8,0.577,2.1,4.1,6.2,0.8,1.3,0.5,0.5,0.8,1.9,1.9,7.6,-3

Test year 2023.

In [91]:
data_test_2023 = data_test[data_test['YEAR_MVP'] == 2023]
data_test_2023.drop('YEAR_MVP', axis=1, inplace=True)

C:\Users\dcast\AppData\Local\Temp\ipykernel_9360\1874860387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_2023.drop('YEAR_MVP', axis=1, inplace=True)


In [92]:
X_test_2023 = data_test_2023.drop('Share', axis=1)
y_test_2023 = data_test_2023['Share']

In [93]:
X_test_2023_tr = pd.DataFrame(scaler.transform(X_test_2023), index=X_test_2023.index, columns=X_test_2023.columns)

In [94]:
pred_2023 = best_rf_model.predict(X_test_2023_tr)

In [95]:
pred_2023 = pd.Series(pred_2023, index=X_test_2023.index)
pred_2023

11427    0.000013
11428    0.000013
11429    0.000013
11430    0.001967
11431    0.000013
           ...   
11961    0.000013
11962    0.002827
11963    0.000013
11964    0.000013
11965    0.024417
Length: 539, dtype: float64

In [96]:
data_or = pd.read_csv('./data/data.csv')
data_2023 = data_or[data_or['YEAR_MVP'] == 2023]

In [ ]:
# data_2006

In [97]:
output_2023 = pd.concat([data_2023, pred_2023], axis=1)

In [98]:
output_2023.sort_values(by=0, ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,0
11826,203999,Nikola Jokic,Nikola,1610612743,DEN,28.0,69,48,21,0.696,33.7,9.4,14.8,0.632,0.8,2.2,0.383,4.9,6.0,0.822,2.4,9.4,11.8,9.8,3.6,1.3,0.7,0.7,2.5,5.7,24.5,9.3,55.7,58,29,50.9,2022-23,203999_2022-23,122.7,124.2,124.2,110.8,111.6,111.6,11.9,12.5,12.5,0.412,2.74,32.1,0.080,0.283,0.186,11.7,11.6,0.660,0.701,0.263,0.268,101.40,100.42,83.68,100.42,0.211,4873,9.4,14.8,1610612743_2022-23,0.646,1,1,2023,203999_2023,0.674,0.748388
11679,203954,Joel Embiid,Joel,1610612755,PHI,29.0,66,43,23,0.652,34.6,11.0,20.1,0.548,1.0,3.0,0.330,10.0,11.7,0.857,1.7,8.4,10.2,4.2,3.4,1.0,1.7,1.4,3.1,7.7,33.1,6.4,56.2,39,1,53.8,2022-23,203954_2022-23,118.3,119.0,119.0,108.6,110.2,110.2,9.7,8.8,8.8,0.233,1.21,12.7,0.057,0.243,0.157,10.5,10.4,0.573,0.655,0.370,0.377,98.41,97.34,81.12,97.34,0.213,4639,11.0,20.1,1610612755_2022-23,0.659,3,2,2023,203954_2023,0.915,0.737856
11593,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,28.0,63,47,16,0.746,32.1,11.2,20.3,0.553,0.7,2.7,0.275,7.9,12.3,0.645,2.2,9.6,11.8,5.7,3.9,0.8,0.8,1.3,3.1,9.0,31.1,5.4,54.8,46,6,52.6,2022-23,203507_2022-23,115.9,116.4,116.4,109.0,109.2,109.2,6.9,7.2,7.2,0.314,1.46,16.2,0.065,0.268,0.171,11.1,11.1,0.572,0.605,0.373,0.383,103.94,103.63,86.36,103.63,0.204,4380,11.2,20.3,1610612749_2022-23,0.707,1,1,2023,203507_2023,0.606,0.702465
11768,1629029,Luka Doncic,Luka,1610612742,DAL,24.0,66,33,33,0.500,36.2,10.9,22.0,0.496,2.8,8.2,0.342,7.8,10.5,0.742,0.8,7.8,8.6,8.0,3.6,1.4,0.5,0.7,2.5,7.3,32.4,1.9,56.8,36,10,55.6,2022-23,1629029_2022-23,116.7,118.1,118.1,114.5,116.0,116.0,2.2,2.1,2.1,0.408,2.24,21.1,0.024,0.224,0.124,9.4,9.4,0.560,0.609,0.368,0.376,98.80,97.63,81.36,97.63,0.202,4874,10.9,22.0,1610612742_2022-23,0.463,11,3,2023,1629029_2023,0.010,0.520925
11665,1628369,Jayson Tatum,Jayson,1610612738,BOS,25.0,74,52,22,0.703,36.9,9.8,21.1,0.466,3.2,9.3,0.350,7.2,8.4,0.854,1.1,7.7,8.8,4.6,2.9,1.1,0.7,0.8,2.2,5.4,30.1,6.4,49.9,31,1,50.2,2022-23,1628369_2022-23,118.2,119.2,119.2,109.5,110.8,110.8,8.7,8.5,8.5,0.205,1.61,14.5,0.029,0.200,0.117,9.0,8.9,0.543,0.607,0.319,0.327,100.77,99.76,83.13,99.76,0.167,5674,9.8,21.1,1610612738_2022-23,0.695,2,1,2023,1628369_2023,0.280,0.447197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11627,1630702,Jaden Hardy,Jaden,1610612742,DAL,20.0,48,20,28,0.417,14.8,3.0,6.9,0.438,1.3,3.3,0.404,1.4,1.6,0.823,0.2,1.6,1.9,1.4,1.0,0.4,0.1,0.6,1.3,1.3,8.8,-0.7,13.5,0,0,14.3,2022-23,1630702_2022-23,112.9,112.5,112.5,115.9,115.6,115.6,-3.1,-3.1,-3.1,0.147,1.41,13.6,0.015,0.106,0.061,9.6,9.6,0.533,0.571,0.248,0.256,101.06,101.37,84.47,101.37,0.090,1502,3.0,6.9,1610612742_2022-23,0.463,11,3,2023,1630702_2023,0.000,0.000013
11625,1631298,Jack White,Jack,1610612743,DEN,25.0,17,13,4,0.765,3.9,0.5,1.1,0.421,0.2,0.5,0.333,0.1,0.2,0.667,0.4,0.6,1.0,0.2,0.1,0.2,0.1,0.1,0.5,0.1,1.2,-1.0,3.6,0,0,3.2,2022-23,1631298_2022-23,96.3,90.4,90.4,107.8,104.5,104.5,-11.5,-14.1,-14.1,0.093,2.00,15.4,0.086,0.177,0.129,7.7,7.6,0.500,0.517,0.133,0.150,92.88,97.41,81.17,97.41,0.087,135,0.5,1.1,1610612743_2022-23,0.646,1,1,2023,1631298_2023,0.000,0.000013
11624,1631133,Jabari Walker,Jabari,1610612757,POR,20.0,56,21,35,0.375,11.1,1.5,3.5,0.419,0.3,1.0,0.286,0.6,0.8,0.756,0.

Model 2.

We will try to optimize for mean absolute error.

In [75]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

grid_search.fit(X_train_tr, y_train)

best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

best_rf_model = grid_search.best_estimator_

neg_mae_train = grid_search.best_score_
mae_train = -neg_mae_train
print("MAE train:", mae_train)


c:\Users\dcast\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Best hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
MAE train: 0.004519001216342962


Test year 2006 (2).

In [84]:
pred_2006_2 = best_rf_model_2.predict(X_test_2006_tr)

In [85]:
pred_2006_2 = pd.Series(pred_2006_2, index=X_test_2006.index)
pred_2006_2

3094    0.000013
3095    0.000013
3096    0.000013
3097    0.000013
3098    0.000013
          ...   
3547    0.000016
3548    0.000013
3549    0.000013
3550    0.000013
3551    0.000042
Length: 458, dtype: float64

In [86]:
output_2006_2 = pd.concat([data_2006, pred_2006_2], axis=1)

In [81]:
# output_2006_2.isna().sum().sum()

In [87]:
output_2006_2.sort_values(by=0, ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,0
3351,977,Kobe Bryant,Kobe,1610612747,LAL,27.0,80,45,35,0.563,41.0,12.2,27.2,0.450,2.3,6.5,0.347,8.7,10.2,0.850,0.9,4.4,5.3,4.5,3.1,1.8,0.4,1.0,2.9,8.1,35.4,4.0,52.0,4,0,51.9,2005-06,977_2005-06,108.4,109.7,109.7,103.7,105.0,105.0,4.7,4.7,4.7,0.228,1.44,11.5,0.022,0.112,0.067,8.0,8.0,0.491,0.559,0.378,0.384,94.36,93.20,77.67,93.20,0.181,6371,12.2,27.2,1610612747_2005-06,0.549,7,3,2006,977_2006,0.386,0.398602
3514,1495,Tim Duncan,Tim,1610612759,SAS,30.0,80,61,19,0.763,34.8,7.2,14.8,0.484,0.0,0.1,0.400,4.2,6.7,0.629,2.9,8.1,11.0,3.2,2.5,0.9,2.0,0.8,2.7,5.6,18.6,6.5,42.7,52,0,38.6,2005-06,1495_2005-06,105.9,106.6,106.6,96.3,97.1,97.1,9.5,9.5,9.5,0.162,1.28,13.5,0.091,0.239,0.167,10.6,10.6,0.485,0.523,0.274,0.277,89.87,89.21,74.35,89.21,0.172,5188,7.2,14.8,1610612759_2005-06,0.768,1,1,2006,1495_2006,0.026,0.396566
3361,2544,LeBron James,LeBron,1610612739,CLE,21.0,79,47,32,0.595,42.5,11.1,23.1,0.480,1.6,4.8,0.335,7.6,10.3,0.738,0.9,6.1,7.0,6.6,3.3,1.6,0.8,0.8,2.3,7.5,31.4,3.5,53.6,21,5,51.4,2005-06,2544_2005-06,107.2,108.2,108.2,103.1,104.4,104.4,4.2,3.8,3.8,0.315,2.00,17.7,0.023,0.150,0.086,8.9,8.8,0.515,0.568,0.329,0.336,92.25,91.25,76.05,91.25,0.192,6404,11.1,23.1,1610612739_2005-06,0.610,3,2,2006,2544_2006,0.550,0.384504
3227,1717,Dirk Nowitzki,Dirk,1610612742,DAL,28.0,81,60,21,0.741,38.1,9.3,19.3,0.480,1.4,3.3,0.406,6.7,7.4,0.901,1.4,7.6,9.0,2.8,1.9,0.7,1.0,0.9,2.0,5.9,26.6,6.3,44.8,35,0,43.2,2005-06,1717_2005-06,111.3,111.9,111.9,102.7,103.3,103.3,8.5,8.6,8.6,0.142,1.45,10.3,0.041,0.209,0.127,7.1,7.1,0.515,0.589,0.293,0.301,90.13,89.64,74.70,89.64,0.205,5774,9.3,19.3,1610612742_2005-06,0.732,2,2,2006,1717_2006,0.435,0.383953
3236,2548,Dwyane Wade,Dwyane,1610612748,MIA,24.0,75,48,27,0.640,38.6,9.3,18.8,0.495,0.2,1.0,0.171,8.4,10.7,0.783,1.4,4.3,5.7,6.7,3.6,1.9,0.8,0.9,2.9,7.8,27.2,6.5,48.7,16,2,45.3,2005-06,2548_2005-06,109.9,110.5,110.5,101.0,102.1,102.1,8.8,8.4,8.4,0.312,1.88,19.8,0.040,0.112,0.078,10.5,10.6,0.499,0.577,0.318,0.321,94.36,93.57,77.98,93.57,0.186,5643,9.3,18.8,1610612748_2005-06,0.634,2,1,2006,2548_2006,0.070,0.346717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,1921,Gordan Giricek,Gordan,1610612762,UTA,29.0,37,18,19,0.486,25.8,4.5,10.4,0.433,0.5,1.6,0.305,1.2,1.5,0.754,0.4,1.5,1.9,1.7,1.7,0.4,0.1,0.4,2.4,1.9,10.6,-2.2,15.4,0,0,15.8,2005-06,1921_2005-06,98.1,99.8,99.8,104.3,103.8,103.8,-6.2,-4.1,-4.1,0.128,1.00,11.8,0.017,0.064,0.040,11.8,11.8,0.457,0.482,0.227,0.230,90.35,89.78,74.82,89.78,0.059,1784,4.5,10.4,1610612762_2005-06,0.500,9,2,2006,1921_2006,0.000,0.000013
3263,101123,Gerald Green,Gerald,1610612738,BOS,20.0,32,13,19,0.406,11.7,2.1,4.3,0.478,0.2,0.6,0.300,0.9,1.2,0.784,0.3,1.0,1.3,0.6,0.7,0.4,0.1,0.1,1.3,0.7,5.2,-1.2,8.5,0,0,8.3,2005-06,101123_2005-06,96.6,95.1,95.1,100.2,97.9,97.9,-3.6,-2.8,-2.8,0.093,0.82,9.2,0.031,0.083,0.059,11.3,11.3,0.500,0.541,0.214,0.219,96.68,98.52,82.10,98.52,0.085,759,2.1,4.3,1610612738_2005-06,0.402,11,3,2006,101123_2006,0.000,0.000013
3262,2809,Gerald Fitch,Gerald,1610612748,MIA,23.0,18,12,6,0.667,13.3,1.7,4.9,0.337,0.4,1.4,0.269,0.9,1.3,0.739,0.4,1.3,1.7,1.8,0.8,0.4,0.3,0.7,1.6,1.1,4.7,-0.3,10.6,0,0,9.9,2005-06,2809_2005-06,99.3,97.3,97.3,9

Test year 2015 (2).

In [83]:
pred_2015_2 = best_rf_model_2.predict(X_test_2015_tr)

In [88]:
pred_2015_2 = pd.Series(pred_2015_2, index=X_test_2015.index)
pred_2015_2

7229    0.000013
7230    0.000013
7231    0.000013
7232    0.000013
7233    0.000075
          ...   
7716    0.000013
7717    0.000013
7718    0.000080
7719    0.000013
7720    0.000013
Length: 492, dtype: float64

In [89]:
output_2015_2 = pd.concat([data_2015, pred_2015_2], axis=1)

In [90]:
output_2015_2.sort_values(by=0, ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,0
7429,201935,James Harden,James,1610612745,HOU,25.0,81,56,25,0.691,36.8,8.0,18.1,0.440,2.6,6.9,0.375,8.8,10.2,0.868,0.9,4.7,5.7,7.0,4.0,1.9,0.7,1.3,2.6,6.7,27.4,4.1,48.6,21,4,47.9,2014-15,201935_2014-15,107.7,109.2,109.2,101.8,103.5,103.5,5.8,5.7,5.7,0.340,1.76,20.8,0.025,0.126,0.075,11.8,11.8,0.511,0.605,0.308,0.313,99.08,97.61,81.35,97.61,0.184,6056,8.0,18.1,1610612745_2014-15,0.683,2,1,2015,201935_2015,0.720,0.369541
7664,201939,Stephen Curry,Stephen,1610612744,GSW,27.0,80,67,13,0.838,32.7,8.2,16.8,0.487,3.6,8.1,0.443,3.9,4.2,0.914,0.7,3.6,4.3,7.7,3.1,2.0,0.2,0.5,2.0,3.7,23.8,11.5,44.1,23,0,43.8,2014-15,201939_2014-15,114.2,114.7,114.7,97.2,98.5,98.5,17.0,16.3,16.3,0.358,2.49,26.4,0.022,0.100,0.063,10.6,10.6,0.594,0.638,0.277,0.283,102.39,101.47,84.56,101.47,0.178,5534,8.2,16.8,1610612744_2014-15,0.817,1,1,2015,201939_2015,0.922,0.220832
7530,2544,LeBron James,LeBron,1610612739,CLE,30.0,69,50,19,0.725,36.1,9.0,18.5,0.488,1.7,4.9,0.354,5.4,7.7,0.710,0.7,5.3,6.0,7.4,3.9,1.6,0.7,0.7,2.0,6.0,25.3,7.8,46.5,22,2,45.0,2014-15,2544_2014-15,112.2,113.8,113.8,102.4,102.8,102.8,9.8,10.9,10.9,0.366,1.88,22.4,0.022,0.147,0.086,11.9,11.9,0.535,0.577,0.318,0.324,94.92,94.06,78.38,94.06,0.171,4889,9.0,18.5,1610612739_2014-15,0.646,2,1,2015,2544_2015,0.425,0.219035
7258,203076,Anthony Davis,Anthony,1610612740,NOP,22.0,68,39,29,0.574,36.1,9.4,17.6,0.535,0.0,0.2,0.083,5.5,6.8,0.805,2.5,7.7,10.2,2.2,1.4,1.5,2.9,0.8,2.1,5.5,24.4,3.9,51.8,42,0,45.6,2014-15,203076_2014-15,108.4,109.9,109.9,103.3,103.8,103.8,5.1,6.1,6.1,0.109,1.57,9.1,0.072,0.215,0.144,5.8,5.8,0.536,0.591,0.272,0.276,94.28,93.40,77.83,93.40,0.192,4765,9.4,17.6,1610612740_2014-15,0.549,8,5,2015,203076_2015,0.156,0.141151
7640,201566,Russell Westbrook,Russell,1610612760,OKC,26.0,67,40,27,0.597,34.4,9.4,22.0,0.426,1.3,4.3,0.299,8.1,9.8,0.835,1.9,5.4,7.3,8.6,4.4,2.1,0.2,1.3,2.7,6.6,28.1,3.7,52.3,31,11,49.9,2014-15,201566_2014-15,109.1,111.0,111.0,105.0,106.3,106.3,4.1,4.7,4.7,0.430,1.96,21.9,0.051,0.151,0.101,11.2,11.2,0.455,0.536,0.368,0.372,100.68,99.23,82.69,99.23,0.187,4769,9.4,22.0,1610612760_2014-15,0.549,9,2,2015,201566_2015,0.271,0.113121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7419,203210,JaMychal Green,JaMychal,1610612763,MEM,25.0,24,10,14,0.417,6.8,1.1,2.0,0.574,0.0,0.3,0.000,0.3,0.4,0.800,0.8,1.1,1.9,0.2,0.6,0.2,0.2,0.1,1.0,0.4,2.6,-1.0,5.8,0,0,5.5,2014-15,203210_2014-15,95.4,95.3,95.3,103.1,101.2,101.2,-7.8,-5.9,-5.9,0.040,0.29,5.7,0.110,0.167,0.137,20.0,20.2,0.574,0.603,0.168,0.171,98.69,99.70,83.08,99.70,0.092,338,1.1,2.0,1610612763_2014-15,0.671,5,2,2015,203210_2015,0.000,0.000013
7418,203960,JaKarr Sampson,JaKarr,1610612755,PHI,22.0,74,17,57,0.230,15.3,2.0,4.7,0.422,0.4,1.7,0.244,0.9,1.3,0.670,0.5,1.7,2.2,1.0,1.0,0.5,0.4,0.4,1.8,1.1,5.2,-4.5,11.0,0,0,10.6,2014-15,203960_2014-15,92.9,91.8,91.8,103.8,104.0,104.0,-11.0,-12.2,-12.2,0.121,1.01,14.2,0.028,0.112,0.068,14.0,14.1,0.467,0.498,0.173,0.178,100.36,100.89,84.07,100.89,0.058,2354,2.0,4.7,1610612755_2014-15,0.220,14,4,2015,203960_2015,0.000,0.000013
7417,2747,JR Smith,JR,1610612739,CLE,29.0,70,38,32,0.543,29.7,4.5,10.8,0.417,2.3,6.1,0.383,0.8,1.0,0.750,0.4,2.7,3.1,2.8,1.4,1.2,0.3,0.2,2.3,

Test year 2023 (2).

In [99]:
pred_2023_2 = best_rf_model_2.predict(X_test_2023_tr)

In [100]:
pred_2023_2 = pd.Series(pred_2023_2, index=X_test_2023.index)
pred_2023_2

11427    0.000013
11428    0.000013
11429    0.000013
11430    0.000412
11431    0.000013
           ...   
11961    0.000013
11962    0.000732
11963    0.000013
11964    0.000013
11965    0.029672
Length: 539, dtype: float64

In [101]:
output_2023_2 = pd.concat([data_2023, pred_2023_2], axis=1)

In [102]:
output_2023_2.sort_values(by=0, ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,0
11826,203999,Nikola Jokic,Nikola,1610612743,DEN,28.0,69,48,21,0.696,33.7,9.4,14.8,0.632,0.8,2.2,0.383,4.9,6.0,0.822,2.4,9.4,11.8,9.8,3.6,1.3,0.7,0.7,2.5,5.7,24.5,9.3,55.7,58,29,50.9,2022-23,203999_2022-23,122.7,124.2,124.2,110.8,111.6,111.6,11.9,12.5,12.5,0.412,2.74,32.1,0.080,0.283,0.186,11.7,11.6,0.660,0.701,0.263,0.268,101.40,100.42,83.68,100.42,0.211,4873,9.4,14.8,1610612743_2022-23,0.646,1,1,2023,203999_2023,0.674,0.772118
11679,203954,Joel Embiid,Joel,1610612755,PHI,29.0,66,43,23,0.652,34.6,11.0,20.1,0.548,1.0,3.0,0.330,10.0,11.7,0.857,1.7,8.4,10.2,4.2,3.4,1.0,1.7,1.4,3.1,7.7,33.1,6.4,56.2,39,1,53.8,2022-23,203954_2022-23,118.3,119.0,119.0,108.6,110.2,110.2,9.7,8.8,8.8,0.233,1.21,12.7,0.057,0.243,0.157,10.5,10.4,0.573,0.655,0.370,0.377,98.41,97.34,81.12,97.34,0.213,4639,11.0,20.1,1610612755_2022-23,0.659,3,2,2023,203954_2023,0.915,0.764706
11593,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,28.0,63,47,16,0.746,32.1,11.2,20.3,0.553,0.7,2.7,0.275,7.9,12.3,0.645,2.2,9.6,11.8,5.7,3.9,0.8,0.8,1.3,3.1,9.0,31.1,5.4,54.8,46,6,52.6,2022-23,203507_2022-23,115.9,116.4,116.4,109.0,109.2,109.2,6.9,7.2,7.2,0.314,1.46,16.2,0.065,0.268,0.171,11.1,11.1,0.572,0.605,0.373,0.383,103.94,103.63,86.36,103.63,0.204,4380,11.2,20.3,1610612749_2022-23,0.707,1,1,2023,203507_2023,0.606,0.710986
11768,1629029,Luka Doncic,Luka,1610612742,DAL,24.0,66,33,33,0.500,36.2,10.9,22.0,0.496,2.8,8.2,0.342,7.8,10.5,0.742,0.8,7.8,8.6,8.0,3.6,1.4,0.5,0.7,2.5,7.3,32.4,1.9,56.8,36,10,55.6,2022-23,1629029_2022-23,116.7,118.1,118.1,114.5,116.0,116.0,2.2,2.1,2.1,0.408,2.24,21.1,0.024,0.224,0.124,9.4,9.4,0.560,0.609,0.368,0.376,98.80,97.63,81.36,97.63,0.202,4874,10.9,22.0,1610612742_2022-23,0.463,11,3,2023,1629029_2023,0.010,0.472976
11665,1628369,Jayson Tatum,Jayson,1610612738,BOS,25.0,74,52,22,0.703,36.9,9.8,21.1,0.466,3.2,9.3,0.350,7.2,8.4,0.854,1.1,7.7,8.8,4.6,2.9,1.1,0.7,0.8,2.2,5.4,30.1,6.4,49.9,31,1,50.2,2022-23,1628369_2022-23,118.2,119.2,119.2,109.5,110.8,110.8,8.7,8.5,8.5,0.205,1.61,14.5,0.029,0.200,0.117,9.0,8.9,0.543,0.607,0.319,0.327,100.77,99.76,83.13,99.76,0.167,5674,9.8,21.1,1610612738_2022-23,0.695,2,1,2023,1628369_2023,0.280,0.464178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11631,203109,Jae Crowder,Jae,1610612749,MIL,32.0,18,13,5,0.722,18.9,2.5,5.2,0.479,1.3,3.1,0.436,0.6,0.7,0.833,0.9,2.9,3.8,1.5,0.4,0.7,0.3,0.1,1.5,0.5,6.9,0.0,16.4,0,0,15.6,2022-23,203109_2022-23,112.2,111.0,111.0,109.7,111.8,111.8,2.5,-0.8,-0.8,0.102,3.86,20.3,0.046,0.134,0.092,5.3,5.3,0.606,0.624,0.125,0.130,105.10,104.63,87.19,104.63,0.096,743,2.5,5.2,1610612749_2022-23,0.707,1,1,2023,203109_2023,0.000,0.000013
11630,1630531,Jaden Springer,Jaden,1610612755,PHI,20.0,16,13,3,0.813,5.6,1.1,2.2,0.486,0.1,0.3,0.400,0.4,0.5,0.750,0.4,0.5,0.9,0.5,0.2,0.4,0.2,0.2,0.9,0.3,2.6,-0.2,6.1,0,0,5.4,2022-23,1630531_2022-23,111.8,108.8,108.8,108.7,110.9,110.9,3.1,-2.1,-2.1,0.129,2.67,16.0,0.063,0.091,0.077,6.0,6.1,0.514,0.545,0.188,0.195,103.64,104.05,86.71,104.05,0.085,194,1.1,2.2,1610612755_2022-23,0.659,3,2,2023,1630531_2023,0.000,0.000013
11629,1630183,Jaden McDaniels,Jaden,1610612750,MIN,22.0,79,41,38,0.519,30.6,4.7,9.1,0.517,1.4,3.4,0.398,1.3,1.8,0.736,1.2,